# Machine Learning Potential Energy Surface example
See sGDML [documentation](http://www.sgdml.org/#code)

In [1]:
import numpy as np
from sgdml.predict import GDMLPredict
from sgdml.utils import io

In [17]:
model = np.load('H2O-QChem-train1200-sym2.npz')
gdml = GDMLPredict(model)

r,_ = io.read_xyz('data_h2o/H2O.xyz')
e,f = gdml.predict(r)

In [18]:
r


array([[ 0.145928 , -0.104312 ,  0.926277 ,  0.0382234,  0.0198018,
        -0.0503268, -0.75256  ,  0.574674 , -0.127562 ]])

In [19]:
e


array([0.00103217])

In [20]:
f

array([[ 0.00323622, -0.00224524, -0.00019206,  0.00177477, -0.00147296,
         0.00470431, -0.00501099,  0.00371821, -0.00451226]])

## Now use Atomic Simulation Environment

In [6]:
!pip install sgdml[ase]

  Using cached ase-3.22.1-py3-none-any.whl (2.2 MB)


In [7]:
from sgdml.intf.ase_calc import SGDMLCalculator

from ase.io import read
from ase.optimize import QuasiNewton
from ase.md.velocitydistribution import (MaxwellBoltzmannDistribution, Stationary, ZeroRotation)
from ase.md.verlet import VelocityVerlet
from ase import units
from IPython.display import HTML
from  ase.build import  molecule

### Function to create html to display molecule

In [8]:
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

In [21]:
model_path = 'H2O-QChem-train1200-sym2.npz'
calc = SGDMLCalculator(model_path)

mol = read('data_h2o/H2O.xyz')
mol.set_calculator(calc)

[WARN] Please remember to specify the proper conversion factors, if your model does not use
       'kcal/mol' and 'Ang' as units.


In [22]:
h2o_html = atoms_to_html(mol)
HTML(h2o_html)

In [23]:
# do a quick geometry relaxation
qn = QuasiNewton(mol)
qn.run(1e-4, 100)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 20:00:43        0.000045        0.0003
BFGSLineSearch:    1[  4] 20:00:44        0.000044        0.0003
BFGSLineSearch:    2[  5] 20:00:44        0.000040        0.0002
BFGSLineSearch:    3[  6] 20:00:44        0.000040        0.0001
BFGSLineSearch:    4[  9] 20:00:44        0.000040        0.0001
BFGSLineSearch:    5[ 13] 20:00:45        0.000038        0.0001


True

### Vibrational Frequencies

In [24]:
from ase.vibrations import Vibrations

In [25]:
vib = Vibrations(mol)
vib.run()
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0   96.3     776.4
  1  114.8     925.7
  2  115.2     929.2
  3  119.6     964.6
  4  259.1    2089.5
  5  407.4    3285.5
  6  474.6    3828.2
  7  487.9    3935.5
  8  518.3    4180.5
---------------------
Zero-point energy: 1.297 eV


In [26]:
qn.run(1e-5, 100)

BFGSLineSearch:    6[ 14] 20:00:58        0.000038        0.0001
BFGSLineSearch:    7[ 16] 20:00:58        0.000038        0.0001
BFGSLineSearch:    8[ 19] 20:00:58        0.000038        0.0001
BFGSLineSearch:    9[ 21] 20:00:58        0.000038        0.0000
BFGSLineSearch:   10[ 22] 20:00:58        0.000038        0.0000


True

### Water IR spectrum
[water vibrations](https://physicsopenlab.org/2022/01/08/water-molecule-vibrations-with-raman-spectroscopy/)

![Water Vibrational Frequency]

In [27]:
vib.run()
vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0   96.3     776.4
  1  114.8     925.7
  2  115.2     929.2
  3  119.6     964.6
  4  259.1    2089.5
  5  407.4    3285.5
  6  474.6    3828.2
  7  487.9    3935.5
  8  518.3    4180.5
---------------------
Zero-point energy: 1.297 eV


In [28]:
vib.get_frequencies(method='standard', direction='central')

array([ 776.42810141+0.j,  925.68303618+0.j,  929.19317637+0.j,
        964.61718577+0.j, 2089.54379873+0.j, 3285.51254529+0.j,
       3828.21045288+0.j, 3935.52854336+0.j, 4180.51897283+0.j])

#### Compare to reported frequencies: [NIST](https://cccbdb.nist.gov/expvibs1x.asp)

### Now display optimized structure

In [ ]:
mol.get_chemical_symbols()

In [ ]:
mol.get_all_distances()

In [ ]:
mol.get_dihedral(1,2,8,3)

In [ ]:
mol.get_distance(2,8) # O-H distance

In [ ]:
mol.get_potential_energy()

In [ ]:
mol.set_distance(2,8,1.7) # O-H distance

In [ ]:
mol.get_potential_energy()

In [ ]:
import numpy as np
en = np.array([])
dist =  np.array([])
for r in np.arange(0.8,4.0,0.01):
    mol.set_distance(2,8,r) # O-H distance
    dist = np.append(dist,mol.get_distance(2,8))
    en = np.append(en,mol.get_potential_energy())

In [ ]:
en = en - min(en) # Remove minimum energy offset

In [ ]:
import plotly.express as px

fig = px.line(x=dist, y=[en])

fig.show()

In [ ]:
mol.get_angle(1,2,8)

In [ ]:
mol.set_angle(1,2,8, 20)
mol.set_distance(2,8, 0.97) # O-H distance

In [ ]:
ena = np.array([])
ang = np.array([])
for a in np.arange(100,180,1):
    mol.set_angle(1,2,8,a) # O-H distance
    ang = np.append(ang,mol.get_angle(1,2,8))
    ena = np.append(ena,mol.get_potential_energy())

In [ ]:
ena = ena - min(ena) # Remove minimum energy offset

In [ ]:
fig = px.line(x=ang, y=[ena])

fig.show()

In [ ]:
max(ena)

## Now scan both bond and angle

In [ ]:
from ase.visualize import view

In [ ]:
from ase.io import write
write('molecule.png', mol)

### 3D Plot concept
Need grid of data

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Make data.
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
# A StrMethodFormatter is used automatically
ax.zaxis.set_major_formatter('{x:.02f}')

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
from mpl_toolkits.mplot3d.axes3d import get_test_data

In [ ]:
X, Y, Z = get_test_data(0.05)

In [ ]:
Y

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator

vel=np.random.random((21,30))
#grid old
x=np.arange(0,21,1)
y=np.arange(0,30,1)
grid_old=(x,y)

#grid new
# the limits of the interpolated x and y val have to be less than the original grid
x_new=np.arange(0.1,19.9,0.1)
y_new=np.arange(0.1,28.9,0.1)
grid_new = np.meshgrid(x_new, y_new)
grid_flattened = np.transpose(np.array([k.flatten() for k in grid_new]))

#Interpolation onto a finer grid
grid_interpol = RegularGridInterpolator(grid_old,vel,method='linear')
vel_interpol = grid_interpol(grid_flattened)

#Unflatten the interpolated velocities and store into a new variable.
index=0
vel_new=np.zeros((len(x_new),len(y_new)))
for i in  range(len(x_new)):
    for j in range(len(y_new)):
        vel_new[i,j] =vel_interpol[index]
        index+=1

fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
surf=ax.plot_surface(grid_new[0],grid_new[1],vel_new.T, cmap="RdBu") 
fig.set_size_inches(10,10) 
plt.show()

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.interpolate
from mpl_toolkits.mplot3d import Axes3D

x = np.random.random(10)
y = np.random.random(10)
z = np.random.random(10)

spline = sp.interpolate.Rbf(x,y,z,function='thin-plate')

xi = np.linspace(x.min(), x.max(), 50)
yi = np.linspace(y.min(), y.max(), 50)
xi, yi = np.meshgrid(xi, yi)

zi = spline(xi,yi)

fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(xi,yi,zi)
plt.show()

## Molecular Dynamics

In [ ]:
from ase.md.velocitydistribution import (MaxwellBoltzmannDistribution,Stationary, ZeroRotation)
from ase.md.verlet import VelocityVerlet
from ase.md import MDLogger
from ase import units
from ase.io.trajectory import Trajectory

In [ ]:
# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(mol, temperature_K=300)
# We want to run MD with constant energy using the VelocityVerlet algorithm.
Stationary(mol)  # zero linear momentum
ZeroRotation(mol)  # zero angular momentum

dyn = VelocityVerlet(mol, 1 * units.fs)  # 5 fs time step.

def printenergy(a=mol):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    bd = a.get_distance(1,8)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

In [ ]:
dyn.attach(printenergy, interval=50)
dyn.attach(MDLogger(dyn, mol, 'md.log', header=True, stress=False,
           peratom=True, mode="a"), interval=10)

# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('moldyn3.traj', 'w', mol)
dyn.attach(traj.write, interval=50)

#### Now run the dynamics

In [24]:
qn.run(1e-5, 100)

KeyboardInterrupt: 

In [ ]:
printenergy(mol)
for i in range(20):
    dyn.run(200)
    printenergy(mol)

Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
